In [ ]:
import pandas as pd
import glob
import json
import matplotlib.pyplot as plt
import os
import pprint
import re
import seaborn as sns

sns.set_style("white", {'axes.grid' : True})

In [ ]:
def join_mask( mask_list,combineor=False ):
    full_mask = None
    for m in mask_list:
        if full_mask is None:
            full_mask = m
        else:
            if combineor:
                full_mask = full_mask | m
            else:
                full_mask = full_mask & m
    return full_mask

def rename_model( m ):
    if m == "faster_rcnn_R_101_FPN_3x":
        return "Faster-RCNN ResNet-101 with FPN"
    elif m == "faster_rcnn_R_50_FPN_3x":
        return "Faster-RCNN ResNet-50 with FPN"
    else:
        return "Unknown"

testsetre = re.compile(r'^(?P<cams>\d+)cameras?-?(?P<camname>[^_]+)?$')
def rename_test_dataset( s, returndict=False ):
    #test-set-1camera-ip8m-h-nw_yolov8
    print(f"GOT IN: {s}")
    m = testsetre.match(s)
    if m is None:
        return "Unknown"
    else:
        gd = m.groupdict()
        pprint.pprint(gd)

        if gd['camname'] is not None:
            camplural = "camera" if int(gd['cams']) == 1 else "cameras"
            if int(gd['cams']) == 1:
                #gd['simplename'] = f"{gd['cams']}{camplural}_{gd['camname'].upper()}"
                gd['simplename'] = f"{gd['camname'].upper()}"
            else:
                if gd['camname'] == "dayonly":
                    gd['simplename'] = f"{gd['cams']} {camplural.capitalize()}, Day"
                else:
                    gd['simplename'] = f"{gd['cams']} {camplural.capitalize()}, Night"
                    
        else:
            gd['simplename'] = f"{gd['cams']} Cameras, All"
        #print(f" returning: {gd['simplename']}")
        if returndict:
            return gd
        else:
            return gd['simplename']

In [ ]:
testdir = "/data/mtmoore/school/CSiML_AI395T/final_project/models/detectron2/test"
#faster_rcnn_R_101_FPN_3x_combined_6batch_512RoI_0
#resultre = re.compile(r'^.+\/detectron2\/validation\/(?P<modelname>.+)_(?P<dataset>[^_]+)_(?P<epochnum>\d+)epochs-val')
resultre = re.compile(r'^.+\/detectron2\/test\/(?P<modelname>.+)_(?P<traindataset>[^_]+)_(?P<testdataset>[^_]+)_final-val')
model_parent = "detectron2"

test_results = []

plot_metrics = ['Precision', 'Recall', 'mAP50', 'mAP50-95', ]

#for valresult in glob.glob(f"{valdir}/*-val1"):
for valresult in glob.glob(f"{testdir}/*-val"):
    print(valresult)
    match = resultre.match(valresult)
    if match is None:
        print(F"didn't match directory name {valresult}")
        continue
    params = match.groupdict()
    new_result = params.copy()
    
    new_result['Model Name'] = rename_model(new_result['modelname'])
        
    if not os.path.exists(f"{valresult}/metrics.json"):
        print(f"{valresult}/metrics.json doesn't exist right now, skipping")
        continue
        
    with open(f"{valresult}/metrics.json") as r:
        metrics = json.load(r)
        if 'bbox' not in metrics:
            print(f"bbox not in metrics for {valresult}, skipping")
            continue
            
        for k, v in metrics['bbox'].items():
            new_result[ k ] = v
        test_results.append(new_result.copy())

In [ ]:
test_df = pd.DataFrame(test_results)

test_df.rename(columns={'traindataset': "Train Dataset", 
                        'testdataset': "Test Dataset",
                        'AP': 'mAP50-95'
                       }, inplace=True
              )

test_df['Test Dataset'] = test_df['Test Dataset'].apply(rename_test_dataset)

#val_df.drop(val_df[ join_mask(mask, combineor=True) ].index, inplace=True)
#val_df.dtypes

if test_df['mAP50-95'].max() > 2:
    test_df['mAP50-95'] = test_df['mAP50-95']/100
print(test_df.dtypes)    
#display(val_df[ (val_df['Camera']=="IP8M-H-NW") & (val_df["Image Size (px)"] == 736) & ( val_df['Model Name'] == "yolov11n") ].sort_values(['Epochs']))

testdataset_order = ["IP8M-G-NW", "IP8M-G-SW", "IP8M-H-NW", "IP8M-H-SW", "4 Cameras, All", "4 Cameras, Day", "4 Cameras, Night"]
test_df['Test Dataset'] = pd.Categorical(test_df['Test Dataset'], categories=testdataset_order, ordered=True)

test_df.sort_values(['Model Name', 'Train Dataset', 'Test Dataset', ], inplace=True)
display(test_df)

In [ ]:
# iterate over cameras
mask_list = []

for traindataset in test_df['Train Dataset'].unique().tolist():
    mask_list.append(test_df['Train Dataset'] == traindataset) 
    print(traindataset)
    # iterate over model
    for model in test_df[join_mask( mask_list )]['Model Name'].unique().tolist(): 
        mask_list.append((test_df['Model Name'] == model))            
        print(f"{traindataset}, {model}")
        current_df = test_df[ join_mask( mask_list ) ]        
        display(current_df)
        fig = plt.figure()
        axs = sns.barplot(data=current_df, x='Test Dataset', y='mAP50-95', hue='Test Dataset') #, palette='colorblind' )
        axs.set_title(f"mAP50-95 Performance for Test Sets\n{traindataset} Training Dataset, Model {model}") 
        #axs.set_ylim([0,1])
        plt.xticks(rotation=45)
        fig.tight_layout()
        plt.savefig(os.path.join("/data/mtmoore/school/CSiML_AI395T/final_project/plots", f"detectron2_test_mAP50-95_{traindataset}_{model}_vary_testset.png".replace(" ", "_")))
        mask_list.pop()
    mask_list.pop()

In [ ]:
#yolov11s/combined 1440px
#yolov11s/combined 2880px
#yolov11s/combined 3840px

# iterate over cameras
#mask_list = []

#mask_list.append(val_df['weights'] == "best")
#mask_list.append(val_df['Train Dataset'] == "combined")
#mask_list.append(val_df['Model Name'] == "yolov11s")


#current_df = val_df[ join_mask(mask_list) ]
#fig = plt.figure()
#axs = sns.barplot(data=current_df, x='Test Dataset', y='mAP50-95', hue='Image Size (px)', palette='colorblind' )
#axs.set_title("mAP50-95 All Test Sets\nCombined Training Dataset, Model yolov11s")
#plt.xticks(rotation=45)
#axs.set_ylim([0,0.4])
#plt.savefig(os.path.join("/data/mtmoore/school/CSiML_AI395T/final_project/plots", f"test_yolov11_allsets_combined_yolov11s.png"))



# iterate over cameras
#mask_list = []

#mask_list.append(val_df['weights'] == "best")
#mask_list.append(val_df['Train Dataset'] == "IP8M-H-NW")
#mask_list.append(val_df['Model Name'] == "yolov11s")
#mask_list.append(val_df['Image Size (px)'] > 736)

#current_df = val_df[ join_mask(mask_list) ]
#fig = plt.figure()
#axs = sns.barplot(data=current_df, x='Test Dataset', y='mAP50-95', hue='Image Size (px)', palette='colorblind' )
#axs.set_title("mAP50-95 All Test Sets\IP8M-H-NW Training Dataset, Model yolov11s")
#plt.xticks(rotation=45)
#axs.set_ylim([0,0.4])
#plt.savefig(os.path.join("/data/mtmoore/school/CSiML_AI395T/final_project/plots", f"test_yolov11_allsets_ip8mhnw_yolov11s.png"))




In [ ]:

#            axs.set_title(f"mAP50-95 {model}/{traindataset}\n{imagesz}px Images ") 
#            #axs.set_ylim([0,1])


#fig, axs = plt.matplotlib.pyplot.subplots(nrows=len(cameras), ncols=1, sharex=True, sharey=False, figsize=(6,8))

#axs[i] = sns.barplot(data=current_df, x='x='Test Dataset', y='mAP50-95', hue='Loss', ax=axs[i])
#for i, imgsz in enumerate(current_df['Image Size (px)'):
        
#            melt_df = pd.melt(current_df, id_vars=['Epoch'], value_vars=plot_metrics, var_name='Loss', value_name='Loss Value')
#            #display(melt_df)
    #axs[i] = sns.barplot(data=current_df, x='x='Test Dataset', y='mAP50-95', hue='Loss', ax=axs[i])
    #axs[i].set_title(f"{camera_name} Dataset") 
        
    #        mask_list.pop()

        
    #    fig.suptitle(f"Training Loss for {model} using {imgsize} pixel Images")
    #    fig.tight_layout()
    #    plt.savefig(os.path.join("/data/mtmoore/school/CSiML_AI395T/final_project/plots", f"training_losses_camera_model_{imgsize}px_{model}.png"))



#for traindataset in val_df['Train Dataset'].unique().tolist():
#    mask_list.append(val_df['Train Dataset'] == traindataset) 
#    # iterate over model
#    for model in val_df[join_mask( mask_list )]['Model Name'].unique().tolist(): 
#        mask_list.append((val_df['Model Name'] == model))
#        #display(current_df)
#        for testdataset in val_df['Test Dataset'].unique().tolist():
#            mask_list.append(val_df['Test Dataset'] == testdataset) 
#            
#            
#        for imagesz in val_df['Image Size (px)'].unique().tolist():
#            print(f"{traindataset}, {model}, {imagesz}")
#            mask_list.append(val_df['Image Size (px)'] == imagesz)
#            current_df = val_df[ join_mask( mask_list ) ]        
#            display(current_df)
#            fig = plt.figure()
#            axs = sns.barplot(data=current_df, x='Test Dataset', y='mAP50-95') #, palette='colorblind' )
#            axs.set_title(f"mAP50-95 {model}/{traindataset}\n{imagesz}px Images ") 
#            #axs.set_ylim([0,1])

#            plt.show()
#            plt.savefig(os.path.join("/data/mtmoore/school/CSiML_AI395T/final_project/plots", f"test_mAP50-95_{traindataset}_{model}_{imagesz}px_vary_testset.png"))
#            plt.close()
#                    
#            mask_list.pop()
#        mask_list.pop()
#    mask_list.pop()



In [ ]:
dnames = [ "test-set-1camera-ip8m-g-nw_coco", 
           "test-set-1camera-ip8m-g-sw_yolov8",  
           "test-set-1camera-ip8m-h-sw_coco",
           "test-set-4cameras-dayonly_coco",
           "test-set-4cameras-nightonly_yolov8",
           "test-set-1camera-ip8m-g-nw_yolov8",  
           "test-set-1camera-ip8m-h-nw_coco", 
           "test-set-1camera-ip8m-h-sw_yolov8",
           "test-set-4cameras-dayonly_yolov8",
           "test-set-4cameras_yolov8",
           "test-set-1camera-ip8m-g-sw_coco",
           "test-set-1camera-ip8m-h-nw_yolov8", 
           "test-set-4cameras_coco",
           "test-set-4cameras-nightonly_coco",
         ]
for d in dnames:
    n = rename_test_dataset(d)
    print(f"{d} -> {n}")